In [ ]:
import os
import sys
sys.path.append('..')
os.environ["CUDA_VISIBLE_DEVICES"] = "0, 3"

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from imageio import imread
from pycocotools import mask as pymask
import seaborn as sns
from tqdm import tqdm
import cv2
import bisect
from typing import Tuple
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_absolute_error

import torch
import torch.nn as nn
from abcmodel.lib.mass import transforms as T
from abcmodel.models.xception import Xception
from abcmodel.models.mass_model import MassModelV3, EnsembleModel

from main import DatasetWrapper

DIR = '/workspace/pig'
DATA_DIR = os.path.join(DIR, 'data/images_20220324/images')

use_bin_class = False

### Data Load

In [ ]:
m_df = pd.read_pickle(os.path.join(DIR, 'data/akiyama/akiyama_20211208.pkl'))

### Model Load

In [ ]:
mobilenet = MassModelV3(
    backbone='mobilenet', input_channels=4, 
    num_features=3)
xception = MassModelV3(
    backbone='xception', input_channels=4, 
    num_features=3)

model = EnsembleModel([mobilenet, xception])

model = torch.nn.DataParallel(model).cuda()
model = model.cuda().eval()

model_path = os.path.join('/workspace/pig/model/', 'val_MRE-0.0482_val_narrow_MRE-0.0371.pth')
model.module.load_state_dict(torch.load(model_path))
model = model.module

### Model Evaluation

In [ ]:
bins = [30., 40., 50., 60., 70., 80., 90., 100., 110.]
bins = bins + [float(i) for i in range(111, 125)] + [125., 130., 140.]

train_df = m_df[m_df["subset"]=="train"]
val_df = m_df[m_df["subset"]=="val"]
# test_df = m_df[m_df["subset"]=="test"]
test_df = test_df = m_df[(m_df['filename'].apply(lambda x: '20211125' in x))]
print('訓練データ数: ', train_df.shape)
print('検証データ数: ', val_df.shape)
print('テストデータ数: ', test_df.shape)

# normalize
weight_mean = train_df['weight'].mean()
weight_std = train_df['weight'].std()

train_dataset = DatasetWrapper(
    dataset=train_df, transforms=T.val_transforms, DATA_DIR=bDATA_DIR)
val_dataset = DatasetWrapper(
    dataset=val_df, transforms=T.val_transforms, DATA_DIR=DATA_DIR)
test_dataset = DatasetWrapper(
    dataset=test_df, transforms=T.val_transforms, DATA_DIR=DATA_DIR)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, num_workers=1, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, num_workers=1, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, num_workers=1, pin_memory=True)

In [ ]:
train_df.columns

In [ ]:
plt.imshow(train_dataset[129][0].numpy()[0])

In [ ]:
train_dataset[0]

### Run Eval

In [ ]:
# eval
model = model.eval()

val_gts = []
val_preds = []
for i, (tensor, feats, gt, _) in tqdm(enumerate(val_loader)):
    _b_size = len(tensor)
    tensor = tensor.cuda()
    feats = feats.cuda()

    tensor = tensor.unsqueeze(1)
    feats = feats.unsqueeze(1)

    tensor_hflip = torch.flip(tensor, dims=(-1,))
    tensor_vflip = torch.flip(tensor, dims=(-2,))
    tensor_vhflip = torch.flip(tensor, dims=(-1, -2))

    # add flip image and ensemble
    tta_tensor = torch.cat([
        tensor, tensor_hflip, tensor_vflip, tensor_vhflip], dim=1)
    tta_feats = feats.repeat_interleave(repeats=4, dim=1)

    assert len(tta_tensor) == len(tta_feats) == _b_size

    tta_tensor = tta_tensor.view(_b_size*4, *tta_tensor.shape[-3:])
    tta_feats = tta_feats.view(_b_size*4, 3)

    assert tta_tensor.dim() == 4

    with torch.no_grad():
        pred = model(tta_tensor, tta_feats)

        pred = pred.view(_b_size, 4, 2, 1)
        pred_mean = pred.mean(axis=(1,2))

        val_gts.extend((gt.flatten()).tolist())
        val_preds.extend((pred_mean.flatten()*weight_std+weight_mean).tolist())

In [ ]:
val_gts = np.array(val_gts)
val_preds = np.array(val_preds)
val_mre = mean_absolute_percentage_error(val_gts, val_preds)

narrow = (val_gts >= 100) & (val_gts <= 120)
val_narrow_mre = mean_absolute_percentage_error(val_gts[narrow], val_preds[narrow])
print(val_mre, ' ', val_narrow_mre)

In [ ]:
norm_val_predicted = []
val_gts = []

model = model.eval()

for i, (tensor, feats, gt, _) in tqdm(enumerate(val_loader)):
    tensor = tensor.cuda()
    feats = feats.cuda()
    gt = gt.cuda()
    tensor_hflip = torch.flip(tensor, dims=(-1,))
    tensor_vflip = torch.flip(tensor, dims=(-2,))
    tensor_vhflip = torch.flip(tensor, dims=(-1,-2))
    tta_tensor = torch.cat([tensor, tensor_hflip, tensor_vflip, tensor_vhflip], dim=0)
    tta_feats = feats.repeat_interleave(repeats=4, dim=0)

    with torch.no_grad():
        if use_bin_class:
            pred, bin_class = model(tta_tensor, tta_feats)
        else:
            pred = model(tta_tensor, tta_feats)

    val_gts.append(gt.item())
    norm_val_predicted.append(pred.mean().item())

val_predicted = list(np.array(norm_val_predicted)*weight_std+weight_mean)
val_predicted = np.clip(val_predicted, 30, 150)
print('検証結果: ', mean_absolute_percentage_error(val_gts, val_predicted))

### Run Test

In [ ]:
norm_test_predicted = []
test_gts = []

for i, (tensor, feats, gt, _) in tqdm(enumerate(test_loader)):
    tensor = tensor.cuda()
    feats = feats.cuda()
    gt = gt.cuda()
    tensor_hflip = torch.flip(tensor, dims=(-1,))
    tensor_vflip = torch.flip(tensor, dims=(-2,))
    tensor_vhflip = torch.flip(tensor, dims=(-1,-2))
    tta_tensor = torch.cat([tensor, tensor_hflip, tensor_vflip, tensor_vhflip], dim=0)
    tta_feats = feats.repeat_interleave(repeats=4, dim=0)

    with torch.no_grad():
        pred = model(tta_tensor, tta_feats)

    test_gts.append(gt.item())
    norm_test_predicted.append(pred.mean().item())
    
test_predicted = list(np.array(norm_test_predicted)*weight_std+weight_mean)
test_predicted = np.clip(test_predicted, 30, 150)

In [ ]:
test_gts = np.array(test_gts)
test_predicted = np.array(test_predicted)
test_mre = mean_absolute_percentage_error(test_gts, test_predicted)

narrow = (test_gts >= 100) & (test_gts <= 120)
test_narrow_mre = mean_absolute_percentage_error(test_gts[narrow], test_predicted[narrow])
print(test_mre, ' ', test_narrow_mre)

In [ ]:
print('検証結果: ', mean_absolute_percentage_error(val_gts, val_predicted))
print('テスト結果: ', mean_absolute_percentage_error(test_gts, test_predicted))

In [ ]:
val_df.loc[:, 'GT'] = val_gts
val_df.loc[:, 'pred'] = val_predicted
val_df.loc[:, 'error'] = (val_df['pred'] - val_df['GT']) / val_df['GT']
# val_df = val_df.reset_index()

test_df.loc[:, 'GT'] = test_gts
test_df.loc[:, 'pred'] = test_predicted
test_df.loc[:, 'error'] = (test_df['pred'] - test_df['GT']) / test_df['GT']
# test_df = test_df.reset_index()

In [ ]:
old_idx = list(val_df[val_df['posture'] != 'None'].index)
new_idx = list(val_df[val_df['posture'] == 'None'].index)
print(len(old_idx), ' ', len(new_idx))

print('旧データの検証誤差: ', mean_absolute_percentage_error(val_df.loc[old_idx]['GT'].values, val_df.loc[old_idx]['pred'].values))
print('新データの検証誤差: ', mean_absolute_percentage_error(val_df.loc[new_idx]['GT'].values, val_df.loc[new_idx]['pred'].values))

test_df[['loc', 'UID', 'GT', 'pred']].sort_values(by='loc').to_csv('JF_test1125.csv', index=False)

In [ ]:
old_idx = list(test_df[test_df['posture'] != 'None'].index)
new_idx = list(test_df[test_df['posture'] == 'None'].index)
print(len(old_idx), ' ', len(new_idx))

print('旧データのテスト誤差: ', mean_absolute_percentage_error(test_df.loc[old_idx]['GT'].values, test_df.loc[old_idx]['pred'].values))
print('新データのテスト誤差: ', mean_absolute_percentage_error(test_df.loc[new_idx]['GT'].values, test_df.loc[new_idx]['pred'].values) )

### Plot result

#### Eval data

In [ ]:
for posture in ['lying', 'standing', 'sitting']:
    idx = list(val_df[val_df['posture'] ==posture].index)
    plt.scatter(val_df['GT'].values[idx], val_df['error'].values[idx], label=posture)

# plt.ylim(-0.5, 0.5)
    
plt.legend()
plt.xlabel('GT')
plt.ylabel('(pred - GT) / GT')

In [ ]:
for posture in ['None']:
    idx = list(val_df[val_df['posture'] ==posture].index)
    plt.scatter(val_df['GT'].values[idx], val_df['error'].values[idx], label=posture, color='red')

# plt.ylim(-0.3, 0.7)
    
plt.legend()
plt.xlabel('GT')
plt.ylabel('(pred - GT) / GT')

#### Test data

In [ ]:
for posture in ['lying', 'standing', 'sitting']:
    idx = list(test_df[test_df['posture'] == posture].index)
    plt.scatter(test_df['GT'].values[idx], test_df['error'].values[idx], label=posture)

# plt.ylim(-0.5, 0.5)
    
plt.legend()
plt.xlabel('GT')
plt.ylabel('(pred - GT) / GT')

In [ ]:
for posture in ['lying', 'standing', 'sitting', 'None']:
    idx = list(test_df[test_df['posture'] == posture].index)
    if posture == 'None':
        posture = 'new data'
    plt.scatter(test_df['GT'].values[idx], test_df['error'].values[idx], label=posture)

# plt.ylim(-0.5, 0.7)
    
plt.legend()
plt.xlabel('GT')
plt.ylabel('(pred - GT) / GT')

In [ ]:
for posture in ['None']:
    idx = list(test_df[test_df['posture'] == posture].index)
    plt.scatter(test_df['GT'].values[idx], test_df['error'].values[idx], label=posture, color='red')

# plt.ylim(-0.5, 0.7)
    
plt.legend()
plt.xlabel('GT')
plt.ylabel('(pred - GT) / GT')

### Error of each bin

#### Val data

In [ ]:
for i,b in enumerate(bins[0:-1]):
    left = bins[i]
    right = bins[i+1]
    print(f'{left} ~ {right}:', np.mean(val_df[(left <= val_df['GT']) & (val_df['GT'] < right)]['error'].abs())*100)

#### Test data

In [ ]:
for i,b in enumerate(bins[0:-1]):
    left = bins[i]
    right = bins[i+1]
    print(f'{left} ~ {right}:', np.mean(test_df[(left <= test_df['GT']) & (test_df['GT'] < right)]['error'].abs())*100)

In [ ]:
for i,b in enumerate(bins[0:-1]):
    left = bins[i]
    right = bins[i+1]
    print(f'{left} ~ {right}:', np.max(test_df[(left <= test_df['GT']) & (test_df['GT'] < right)]['error'].abs())*100)

### Check bad performance data

In [ ]:
test_df[(test_df['error']*100).abs() < 1][['GT', 'pred', 'error']]

In [ ]:
pig = test_df.iloc[36]

img = imread(os.path.join(DATA_DIR, pig.filename))
x0, y0, w, h = pig.bbox
img = img[y0:y0 + h, x0:x0 + w] 

mask = pymask.decode(pig.polygon._rle)
mask = mask[y0:y0 + h, x0:x0 + w]


depth = imread(os.path.join(
    DATA_DIR, pig.filename.replace('rgb', 'depth').replace('.jpg', '.png')))

depth = cv2.resize(depth, (640, 480))
depth[depth > pig.floor_depth] = pig.floor_depth
depth = depth[y0:y0 + h, x0:x0 + w]
depth = np.where(depth != 0, pig.floor_depth - depth.astype(int), 0)

mask = pymask.decode(pig.polygon._rle)
mask = mask[y0:y0 + h, x0:x0 + w]

# depth正規化を追加 
depth[depth == 0] = depth.mean()
depth *= mask
_sigma = depth.mean() + 2*depth.std()
depth[depth> _sigma] = depth[depth > 0].mean()
# depth正規化追加


fix, ax = plt.subplots(figsize=(8, 4), ncols=3);
print(f'GT: {pig.GT}, pred: {pig.pred}')
ax[0].imshow(img);
ax[1].imshow(mask);
ax[2].imshow(depth);

In [ ]:
pig